# Predict Churn on "Telecom Churn Case Study Hackathon" Dataset

Predict churning customers for a Telecom company based on temporal behaviour

In [66]:
# Import libraries and setup
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/telecom-churn/data_dictionary.csv
/kaggle/input/telecom-churn/sample.csv
/kaggle/input/telecom-churn/starter-notebook - Labs 1.ipynb
/kaggle/input/telecom-churn/train.csv
/kaggle/input/telecom-churn/test.csv


In [67]:
#Import dataset and combine the train and test for cleaning part
train = pd.read_csv('/kaggle/input/telecom-churn/train.csv')
test = pd.read_csv('/kaggle/input/telecom-churn/test.csv')
df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])
df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,ind,churn_probability
0,69999,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,91.882,65.330,...,0,NaN,NaN,NaN,1692,0.00,0.00,0.00,test,NaN
1,70000,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,414.168,515.568,...,0,NaN,NaN,NaN,2533,0.00,0.00,0.00,test,NaN
2,70001,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,329.844,434.884,...,0,NaN,NaN,NaN,277,525.61,758.41,241.84,test,NaN
3,70002,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,43.550,171.390,...,0,NaN,NaN,NaN,1244,0.00,0.00,0.00,test,NaN
4,70003,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,306.854,406.289,...,0,NaN,NaN,NaN,462,0.00,0.00,0.00,test,NaN


In [30]:
# Check statistics
df.describe()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,99999.000000,99999.0,98981.0,98981.0,98981.0,99999.000000,99999.000000,99999.000000,96062.000000,96140.000000,...,99999.000000,99999.000000,25153.000000,25571.000000,26339.000000,99999.000000,99999.000000,99999.000000,99999.000000,69999.000000
mean,49999.000000,109.0,0.0,0.0,0.0,282.987358,278.536648,279.154731,132.395875,133.670805,...,0.080401,0.084501,0.914404,0.908764,0.890808,1219.854749,68.170248,66.839062,60.021204,0.101887
std,28867.369122,0.0,0.0,0.0,0.0,328.439770,338.156291,344.474791,297.207406,308.794148,...,0.628334,0.660234,0.279772,0.287950,0.311885,954.733842,267.580450,271.201856,253.938223,0.302502
min,0.000000,109.0,0.0,0.0,0.0,-2258.709000,-2014.045000,-945.808000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000,0.000000
25%,24999.500000,109.0,0.0,0.0,0.0,93.411500,86.980500,84.126000,7.380000,6.660000,...,0.000000,0.000000,1.000000,1.000000,1.000000,467.000000,0.000000,0.000000,0.000000,0.000000
50%,49999.000000,109.0,0.0,0.0,0.0,197.704000,191.640000,192.080000,34.310000,32.330000,...,0.000000,0.000000,1.000000,1.000000,1.000000,863.000000,0.000000,0.000000,0.000000,0.000000
75%,74998.500000,109.0,0.0,0.0,0.0,371.060000,365.344500,369.370500,118.740000,115.595000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1807.500000,0.000000,0.000000,0.000000,0.000000
max,99998.000000,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,...,35.000000,41.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.210000,1.000000


COLUMN DESC FROM DATASET:

*.6   =   KPI for the month of June  
*.7   =   KPI for the month of July  
*.8   =   KPI for the month of August  
2G   =   G network  
3G   =   G network  
AMT   =   Amount in local currency  
AON   =   Age on network  number of days the customer is using the operator T network  
ARPU   =   Average revenue per user  
AV   =   Average  
CIRCLE_ID   =   Telecom circle area to which the customer belongs to  
DATA   =   Mobile internet  
FB_USER   =   Service scheme to avail services of Facebook and similar social networking sites  
IC   =   Incoming calls  
ISD   =   ISD calls  
LOC   =   Local calls  within same telecom circle  
MAX   =   Maximum  
MONTHLY   =   Service schemes with validity equivalent to a month  
MOU   =   Minutes of usage  voice calls  
NIGHT   =   Scheme to use during specific night hours only  
NUM   =   Number  
OFFNET   =   All kind of calls outside the operator T network  
OG   =   Outgoing calls  
ONNET   =   All kind of calls within the same operator network  
PCK   =   Prepaid service schemes called  PACKS  
RECH   =   Recharge  
ROAM   =   Indicates that customer is in roaming zone during the call  
SACHET   =   Service schemes with validity smaller than a month  
SPL   =   Special calls  
STD   =   STD calls  outside the calling circle  
T2C   =   Operator T to its own call center  
T2F   =   Operator T to fixed lines of T  
T2M   =   Operator T to other operator mobile  
T2O   =   Operator T to other operator fixed line  
T2T   =   Operator T to T ie within same operator mobile to mobile  
VBC   =   Volume based cost  when no specific scheme is not purchased and paid as per usage  
VOL   =   Mobile internet usage volume in MB  

# Data Cleaning

1. UNUSED COLUMN
- id = only display unique id
- circle_id = have same value for every rows

2. CHECK COLUMN WITH 0 VALUES
- check column with 0 on all rows then drop the column

3. CHECK NA

In [59]:
# Check column with 0 value
df.describe().loc[:, df.describe().loc['mean'] == 0]

,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8
count,98981.0,98981.0,98981.0,96062.0,96140.0,94621.0,96062.0,96140.0,94621.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
# Remove 9 column with zero data + 2 unused column
df.drop(columns=['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou',
                 'std_og_t2c_mou_6','std_og_t2c_mou_7','std_og_t2c_mou_8',
                 'std_ic_t2o_mou_6','std_ic_t2o_mou_7','std_ic_t2o_mou_8',
                 'id','circle_id'], axis = 1, inplace = True)

print(df.shape,'With 11 columns dropped, left with only 162 columns')

(99999, 162) With 11 columns dropped, left with only 162 columns


In [73]:
# Check NA
df.isna().sum().sort_values(ascending=False)[lambda x: x > 0].to_numpy()

array([], dtype=int64)

# Data Feature Engineering

1. CREATE NEW DATA POINT TO CAPTURE DAY DIFFERENCE BETWEEN LAST RECHARGE AND LAST DATE OF MONTH
- Used date_of_last_rech and date_of_last_rech_data column
- Then drop the date column for modelling

2. LINKED 3 MONTH DATA TO CAPTURE TREND
- Add 3 feature for each 52 column monthly trend. Diff from 1st to 2nd, diff 2nd to 3rd and diff 1st to 3rd 

In [69]:
# Transform date column into difference date to last day of month its month, and drop date column after

date_columns = ['date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8',
                'date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8',
               'last_date_of_month_6','last_date_of_month_7','last_date_of_month_8']

df[date_columns] = df[date_columns].apply(pd.to_datetime)

df['diff_date_of_last_rech_6'] = (df['last_date_of_month_6'] - df['date_of_last_rech_6']).dt.days 
df['diff_date_of_last_rech_7'] = (df['last_date_of_month_7'] - df['date_of_last_rech_7']).dt.days  
df['diff_date_of_last_rech_8'] = (df['last_date_of_month_8'] - df['date_of_last_rech_8']).dt.days 

df['diff_date_of_last_rech_data_6'] = (df['last_date_of_month_6'] - df['date_of_last_rech_data_6']).dt.days  
df['diff_date_of_last_rech_data_7'] = (df['last_date_of_month_7'] - df['date_of_last_rech_data_7']).dt.days  
df['diff_date_of_last_rech_data_8'] = (df['last_date_of_month_8'] - df['date_of_last_rech_data_8']).dt.days 

df.drop(columns=date_columns,axis=1, inplace = True)
df.shape


(99999, 159)

In [70]:
# Prepare data before transforming
df.rename(columns={'jun_vbc_3g':'vbc_3g_6',
           'jul_vbc_3g':'vbc_3g_7',
           'aug_vbc_3g':'vbc_3g_8'},inplace=True)

df.fillna(0,inplace=True)

In [71]:
# Create feature engineering for 52 monthly trend column
df['arpu_67'] = df['arpu_7'] - df['arpu_6']
df['arpu_78'] = df['arpu_8'] - df['arpu_7']
df['arpu_68'] = df['arpu_8'] - df['arpu_6']
df['onnet_mou_67'] = df['onnet_mou_7'] - df['onnet_mou_6']
df['onnet_mou_78'] = df['onnet_mou_8'] - df['onnet_mou_7']
df['onnet_mou_68'] = df['onnet_mou_8'] - df['onnet_mou_6']
df['offnet_mou_67'] = df['offnet_mou_7'] - df['offnet_mou_6']
df['offnet_mou_78'] = df['offnet_mou_8'] - df['offnet_mou_7']
df['offnet_mou_68'] = df['offnet_mou_8'] - df['offnet_mou_6']
df['roam_ic_mou_67'] = df['roam_ic_mou_7'] - df['roam_ic_mou_6']
df['roam_ic_mou_78'] = df['roam_ic_mou_8'] - df['roam_ic_mou_7']
df['roam_ic_mou_68'] = df['roam_ic_mou_8'] - df['roam_ic_mou_6']
df['roam_og_mou_67'] = df['roam_og_mou_7'] - df['roam_og_mou_6']
df['roam_og_mou_78'] = df['roam_og_mou_8'] - df['roam_og_mou_7']
df['roam_og_mou_68'] = df['roam_og_mou_8'] - df['roam_og_mou_6']
df['loc_og_t2t_mou_67'] = df['loc_og_t2t_mou_7'] - df['loc_og_t2t_mou_6']
df['loc_og_t2t_mou_78'] = df['loc_og_t2t_mou_8'] - df['loc_og_t2t_mou_7']
df['loc_og_t2t_mou_68'] = df['loc_og_t2t_mou_8'] - df['loc_og_t2t_mou_6']
df['loc_og_t2m_mou_67'] = df['loc_og_t2m_mou_7'] - df['loc_og_t2m_mou_6']
df['loc_og_t2m_mou_78'] = df['loc_og_t2m_mou_8'] - df['loc_og_t2m_mou_7']
df['loc_og_t2m_mou_68'] = df['loc_og_t2m_mou_8'] - df['loc_og_t2m_mou_6']
df['loc_og_t2f_mou_67'] = df['loc_og_t2f_mou_7'] - df['loc_og_t2f_mou_6']
df['loc_og_t2f_mou_78'] = df['loc_og_t2f_mou_8'] - df['loc_og_t2f_mou_7']
df['loc_og_t2f_mou_68'] = df['loc_og_t2f_mou_8'] - df['loc_og_t2f_mou_6']
df['loc_og_t2c_mou_67'] = df['loc_og_t2c_mou_7'] - df['loc_og_t2c_mou_6']
df['loc_og_t2c_mou_78'] = df['loc_og_t2c_mou_8'] - df['loc_og_t2c_mou_7']
df['loc_og_t2c_mou_68'] = df['loc_og_t2c_mou_8'] - df['loc_og_t2c_mou_6']
df['loc_og_mou_67'] = df['loc_og_mou_7'] - df['loc_og_mou_6']
df['loc_og_mou_78'] = df['loc_og_mou_8'] - df['loc_og_mou_7']
df['loc_og_mou_68'] = df['loc_og_mou_8'] - df['loc_og_mou_6']
df['std_og_t2t_mou_67'] = df['std_og_t2t_mou_7'] - df['std_og_t2t_mou_6']
df['std_og_t2t_mou_78'] = df['std_og_t2t_mou_8'] - df['std_og_t2t_mou_7']
df['std_og_t2t_mou_68'] = df['std_og_t2t_mou_8'] - df['std_og_t2t_mou_6']
df['std_og_t2m_mou_67'] = df['std_og_t2m_mou_7'] - df['std_og_t2m_mou_6']
df['std_og_t2m_mou_78'] = df['std_og_t2m_mou_8'] - df['std_og_t2m_mou_7']
df['std_og_t2m_mou_68'] = df['std_og_t2m_mou_8'] - df['std_og_t2m_mou_6']
df['std_og_t2f_mou_67'] = df['std_og_t2f_mou_7'] - df['std_og_t2f_mou_6']
df['std_og_t2f_mou_78'] = df['std_og_t2f_mou_8'] - df['std_og_t2f_mou_7']
df['std_og_t2f_mou_68'] = df['std_og_t2f_mou_8'] - df['std_og_t2f_mou_6']
df['std_og_mou_67'] = df['std_og_mou_7'] - df['std_og_mou_6']
df['std_og_mou_78'] = df['std_og_mou_8'] - df['std_og_mou_7']
df['std_og_mou_68'] = df['std_og_mou_8'] - df['std_og_mou_6']
df['isd_og_mou_67'] = df['isd_og_mou_7'] - df['isd_og_mou_6']
df['isd_og_mou_78'] = df['isd_og_mou_8'] - df['isd_og_mou_7']
df['isd_og_mou_68'] = df['isd_og_mou_8'] - df['isd_og_mou_6']
df['spl_og_mou_67'] = df['spl_og_mou_7'] - df['spl_og_mou_6']
df['spl_og_mou_78'] = df['spl_og_mou_8'] - df['spl_og_mou_7']
df['spl_og_mou_68'] = df['spl_og_mou_8'] - df['spl_og_mou_6']
df['og_others_67'] = df['og_others_7'] - df['og_others_6']
df['og_others_78'] = df['og_others_8'] - df['og_others_7']
df['og_others_68'] = df['og_others_8'] - df['og_others_6']
df['total_og_mou_67'] = df['total_og_mou_7'] - df['total_og_mou_6']
df['total_og_mou_78'] = df['total_og_mou_8'] - df['total_og_mou_7']
df['total_og_mou_68'] = df['total_og_mou_8'] - df['total_og_mou_6']
df['loc_ic_t2t_mou_67'] = df['loc_ic_t2t_mou_7'] - df['loc_ic_t2t_mou_6']
df['loc_ic_t2t_mou_78'] = df['loc_ic_t2t_mou_8'] - df['loc_ic_t2t_mou_7']
df['loc_ic_t2t_mou_68'] = df['loc_ic_t2t_mou_8'] - df['loc_ic_t2t_mou_6']
df['loc_ic_t2m_mou_67'] = df['loc_ic_t2m_mou_7'] - df['loc_ic_t2m_mou_6']
df['loc_ic_t2m_mou_78'] = df['loc_ic_t2m_mou_8'] - df['loc_ic_t2m_mou_7']
df['loc_ic_t2m_mou_68'] = df['loc_ic_t2m_mou_8'] - df['loc_ic_t2m_mou_6']
df['loc_ic_t2f_mou_67'] = df['loc_ic_t2f_mou_7'] - df['loc_ic_t2f_mou_6']
df['loc_ic_t2f_mou_78'] = df['loc_ic_t2f_mou_8'] - df['loc_ic_t2f_mou_7']
df['loc_ic_t2f_mou_68'] = df['loc_ic_t2f_mou_8'] - df['loc_ic_t2f_mou_6']
df['loc_ic_mou_67'] = df['loc_ic_mou_7'] - df['loc_ic_mou_6']
df['loc_ic_mou_78'] = df['loc_ic_mou_8'] - df['loc_ic_mou_7']
df['loc_ic_mou_68'] = df['loc_ic_mou_8'] - df['loc_ic_mou_6']
df['std_ic_t2t_mou_67'] = df['std_ic_t2t_mou_7'] - df['std_ic_t2t_mou_6']
df['std_ic_t2t_mou_78'] = df['std_ic_t2t_mou_8'] - df['std_ic_t2t_mou_7']
df['std_ic_t2t_mou_68'] = df['std_ic_t2t_mou_8'] - df['std_ic_t2t_mou_6']
df['std_ic_t2m_mou_67'] = df['std_ic_t2m_mou_7'] - df['std_ic_t2m_mou_6']
df['std_ic_t2m_mou_78'] = df['std_ic_t2m_mou_8'] - df['std_ic_t2m_mou_7']
df['std_ic_t2m_mou_68'] = df['std_ic_t2m_mou_8'] - df['std_ic_t2m_mou_6']
df['std_ic_t2f_mou_67'] = df['std_ic_t2f_mou_7'] - df['std_ic_t2f_mou_6']
df['std_ic_t2f_mou_78'] = df['std_ic_t2f_mou_8'] - df['std_ic_t2f_mou_7']
df['std_ic_t2f_mou_68'] = df['std_ic_t2f_mou_8'] - df['std_ic_t2f_mou_6']
df['std_ic_mou_67'] = df['std_ic_mou_7'] - df['std_ic_mou_6']
df['std_ic_mou_78'] = df['std_ic_mou_8'] - df['std_ic_mou_7']
df['std_ic_mou_68'] = df['std_ic_mou_8'] - df['std_ic_mou_6']
df['total_ic_mou_67'] = df['total_ic_mou_7'] - df['total_ic_mou_6']
df['total_ic_mou_78'] = df['total_ic_mou_8'] - df['total_ic_mou_7']
df['total_ic_mou_68'] = df['total_ic_mou_8'] - df['total_ic_mou_6']
df['spl_ic_mou_67'] = df['spl_ic_mou_7'] - df['spl_ic_mou_6']
df['spl_ic_mou_78'] = df['spl_ic_mou_8'] - df['spl_ic_mou_7']
df['spl_ic_mou_68'] = df['spl_ic_mou_8'] - df['spl_ic_mou_6']
df['isd_ic_mou_67'] = df['isd_ic_mou_7'] - df['isd_ic_mou_6']
df['isd_ic_mou_78'] = df['isd_ic_mou_8'] - df['isd_ic_mou_7']
df['isd_ic_mou_68'] = df['isd_ic_mou_8'] - df['isd_ic_mou_6']
df['ic_others_67'] = df['ic_others_7'] - df['ic_others_6']
df['ic_others_78'] = df['ic_others_8'] - df['ic_others_7']
df['ic_others_68'] = df['ic_others_8'] - df['ic_others_6']
df['total_rech_num_67'] = df['total_rech_num_7'] - df['total_rech_num_6']
df['total_rech_num_78'] = df['total_rech_num_8'] - df['total_rech_num_7']
df['total_rech_num_68'] = df['total_rech_num_8'] - df['total_rech_num_6']
df['total_rech_amt_67'] = df['total_rech_amt_7'] - df['total_rech_amt_6']
df['total_rech_amt_78'] = df['total_rech_amt_8'] - df['total_rech_amt_7']
df['total_rech_amt_68'] = df['total_rech_amt_8'] - df['total_rech_amt_6']
df['max_rech_amt_67'] = df['max_rech_amt_7'] - df['max_rech_amt_6']
df['max_rech_amt_78'] = df['max_rech_amt_8'] - df['max_rech_amt_7']
df['max_rech_amt_68'] = df['max_rech_amt_8'] - df['max_rech_amt_6']
df['last_day_rch_amt_67'] = df['last_day_rch_amt_7'] - df['last_day_rch_amt_6']
df['last_day_rch_amt_78'] = df['last_day_rch_amt_8'] - df['last_day_rch_amt_7']
df['last_day_rch_amt_68'] = df['last_day_rch_amt_8'] - df['last_day_rch_amt_6']
df['total_rech_data_67'] = df['total_rech_data_7'] - df['total_rech_data_6']
df['total_rech_data_78'] = df['total_rech_data_8'] - df['total_rech_data_7']
df['total_rech_data_68'] = df['total_rech_data_8'] - df['total_rech_data_6']
df['max_rech_data_67'] = df['max_rech_data_7'] - df['max_rech_data_6']
df['max_rech_data_78'] = df['max_rech_data_8'] - df['max_rech_data_7']
df['max_rech_data_68'] = df['max_rech_data_8'] - df['max_rech_data_6']
df['count_rech_2g_67'] = df['count_rech_2g_7'] - df['count_rech_2g_6']
df['count_rech_2g_78'] = df['count_rech_2g_8'] - df['count_rech_2g_7']
df['count_rech_2g_68'] = df['count_rech_2g_8'] - df['count_rech_2g_6']
df['count_rech_3g_67'] = df['count_rech_3g_7'] - df['count_rech_3g_6']
df['count_rech_3g_78'] = df['count_rech_3g_8'] - df['count_rech_3g_7']
df['count_rech_3g_68'] = df['count_rech_3g_8'] - df['count_rech_3g_6']
df['av_rech_amt_data_67'] = df['av_rech_amt_data_7'] - df['av_rech_amt_data_6']
df['av_rech_amt_data_78'] = df['av_rech_amt_data_8'] - df['av_rech_amt_data_7']
df['av_rech_amt_data_68'] = df['av_rech_amt_data_8'] - df['av_rech_amt_data_6']
df['vol_2g_mb_67'] = df['vol_2g_mb_7'] - df['vol_2g_mb_6']
df['vol_2g_mb_78'] = df['vol_2g_mb_8'] - df['vol_2g_mb_7']
df['vol_2g_mb_68'] = df['vol_2g_mb_8'] - df['vol_2g_mb_6']
df['vol_3g_mb_67'] = df['vol_3g_mb_7'] - df['vol_3g_mb_6']
df['vol_3g_mb_78'] = df['vol_3g_mb_8'] - df['vol_3g_mb_7']
df['vol_3g_mb_68'] = df['vol_3g_mb_8'] - df['vol_3g_mb_6']
df['arpu_3g_67'] = df['arpu_3g_7'] - df['arpu_3g_6']
df['arpu_3g_78'] = df['arpu_3g_8'] - df['arpu_3g_7']
df['arpu_3g_68'] = df['arpu_3g_8'] - df['arpu_3g_6']
df['arpu_2g_67'] = df['arpu_2g_7'] - df['arpu_2g_6']
df['arpu_2g_78'] = df['arpu_2g_8'] - df['arpu_2g_7']
df['arpu_2g_68'] = df['arpu_2g_8'] - df['arpu_2g_6']
df['night_pck_user_67'] = df['night_pck_user_7'] - df['night_pck_user_6']
df['night_pck_user_78'] = df['night_pck_user_8'] - df['night_pck_user_7']
df['night_pck_user_68'] = df['night_pck_user_8'] - df['night_pck_user_6']
df['monthly_2g_67'] = df['monthly_2g_7'] - df['monthly_2g_6']
df['monthly_2g_78'] = df['monthly_2g_8'] - df['monthly_2g_7']
df['monthly_2g_68'] = df['monthly_2g_8'] - df['monthly_2g_6']
df['sachet_2g_67'] = df['sachet_2g_7'] - df['sachet_2g_6']
df['sachet_2g_78'] = df['sachet_2g_8'] - df['sachet_2g_7']
df['sachet_2g_68'] = df['sachet_2g_8'] - df['sachet_2g_6']
df['monthly_3g_67'] = df['monthly_3g_7'] - df['monthly_3g_6']
df['monthly_3g_78'] = df['monthly_3g_8'] - df['monthly_3g_7']
df['monthly_3g_68'] = df['monthly_3g_8'] - df['monthly_3g_6']
df['sachet_3g_67'] = df['sachet_3g_7'] - df['sachet_3g_6']
df['sachet_3g_78'] = df['sachet_3g_8'] - df['sachet_3g_7']
df['sachet_3g_68'] = df['sachet_3g_8'] - df['sachet_3g_6']
df['fb_user_67'] = df['fb_user_7'] - df['fb_user_6']
df['fb_user_78'] = df['fb_user_8'] - df['fb_user_7']
df['fb_user_68'] = df['fb_user_8'] - df['fb_user_6']
df['vbc_3g_67'] = df['vbc_3g_7'] - df['vbc_3g_6']
df['vbc_3g_78'] = df['vbc_3g_8'] - df['vbc_3g_7']
df['vbc_3g_68'] = df['vbc_3g_8'] - df['vbc_3g_6']
df.shape
;

/tmp/ipykernel_126/4275304662.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['max_rech_amt_78'] = df['max_rech_amt_8'] - df['max_rech_amt_7']
/tmp/ipykernel_126/4275304662.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['max_rech_amt_68'] = df['max_rech_amt_8'] - df['max_rech_amt_6']
/tmp/ipykernel_126/4275304662.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

''

In [ ]:
# from sklearn.preprocessing import StandardScaler

# excluded_column = ['ind','churn_probability']
# columns_list = [col for col in df.columns if col != excluded_column]
# scaler = StandardScaler()

# df[columns_list] = scaler.fit_transform(df[columns_list])

# Data Splitting



In [9]:
# Split the train and test after preprocessing
test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]
train.drop(columns=['ind'], axis = 1, inplace = True)
test.drop(columns=['churn_probability','ind'], axis = 1, inplace = True)


print(train.shape)
print(test.shape)

(69999, 308)
(30000, 307)


/tmp/ipykernel_32/1076901766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(columns=['ind'], axis = 1, inplace = True)
/tmp/ipykernel_32/1076901766.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns=['churn_probability','ind'], axis = 1, inplace = True)


In [ ]:
# df_train = train.sample(frac=0.85, random_state=42)
# df_valid = train.drop(df_train.index)

# X_train = df_train.drop('churn_probability', axis=1)
# X_valid = df_valid.drop('churn_probability', axis=1)
# y_train = df_train['churn_probability']
# y_valid = df_valid['churn_probability']



In [10]:
# Split train and validation data for modelling
from sklearn.model_selection import train_test_split

X = train.drop('churn_probability', axis=1)
y = train['churn_probability']

X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                   train_size=0.85,
                                                   stratify=y, random_state=42)

X_train.shape, X_valid.shape

((59499, 307), (10500, 307))

# Machine Learning Model

- Base model build using Random Forest, XGB, Catboost, LGBM model
- Using optuna for hyperparameter tuning
- Combine base model with Stacking Classifier and Voting Classifier


In [ ]:
def model_builder(hp):
    model = keras.Sequential()
    
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=128)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=128)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=128)
    # hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=64)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])
    
    model.add(keras.layers.BatchNormalization(input_shape=[308]))
    model.add(keras.layers.Dense(units=hp_layer_1, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(units=hp_layer_2, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(units=hp_layer_3, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.Dense(units=hp_layer_4, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model


import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                    objective='val_binary_accuracy',
                    max_epochs=42,
                    factor=3,
                    directory='dir',
                    project_name='x',
                    overwrite = True)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    patience=3,
    min_delta=0.001,
    restore_best_weights=True)

In [ ]:
tuner.search(X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=64,
    epochs=21,callbacks=[early_stopping]
            )

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model_hyp = tuner.hypermodel.build(best_hps)



history_hyp = model_hyp.fit(X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=64,
    epochs=66)

In [ ]:
history_df = pd.DataFrame(history_hyp.history)
history_df.loc[1:, ['loss', 'val_loss']].plot()
history_df.loc[1:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

In [11]:
from sklearn.metrics import accuracy_score

X_test = X_valid
y_test = y_valid

In [11]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(3) # Define classifier
knn.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# Training set performance
knn_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set performance
knn_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % knn_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % knn_test_accuracy)

Model performance for Training set
- Accuracy: 0.9414612010285888
----------------------------------
Model performance for Test set
- Accuracy: 0.9015238095238095


In [ ]:
from sklearn.svm import SVC

svm_rbf = SVC(gamma=2, C=1)
# svm_rbf.fit(X_train, y_train)

# Make predictions
y_train_pred = svm_rbf.predict(X_train)
y_test_pred = svm_rbf.predict(X_test)

# Training set performance
svm_rbf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set performance
svm_rbf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % svm_rbf_train_accuracy)
print('----------------------------------')
print('Model performance for Valid set')
print('- Accuracy: %s' % svm_rbf_test_accuracy)

In [14]:
import optuna
from sklearn.tree import DecisionTreeClassifier

def objective(trial):
    x,y = X_train, y_train
    x_test_dt,y_test_dt = X_test,y_test
    
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth',3,21)
    min_samples_split = trial.suggest_int('min_samples_split',2,11)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes',3,26)
    
    dt = DecisionTreeClassifier(criterion=criterion,
                               max_depth=max_depth,
                               min_samples_split=min_samples_split,
                               max_leaf_nodes=max_leaf_nodes) 
    
    dt.fit(x, y) 
        
    y_test_pred = dt.predict(x_test_dt)
    dt_accuracy = accuracy_score(y_test_dt, y_test_pred)  
    
    return dt_accuracy
    
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=99)

[I 2023-07-07 10:51:23,561] A new study created in memory with name: no-name-5aed11c2-e90b-4973-9191-6e4fdfd133ed
[I 2023-07-07 10:51:30,585] Trial 0 finished with value: 0.9379047619047619 and parameters: {'regressor': 'entropy', 'max_depth': 11, 'min_samples_split': 9, 'max_leaf_nodes': 15}. Best is trial 0 with value: 0.9379047619047619.
[I 2023-07-07 10:51:37,561] Trial 1 finished with value: 0.9379047619047619 and parameters: {'regressor': 'entropy', 'max_depth': 10, 'min_samples_split': 6, 'max_leaf_nodes': 14}. Best is trial 0 with value: 0.9379047619047619.
[I 2023-07-07 10:51:43,588] Trial 2 finished with value: 0.9393333333333334 and parameters: {'regressor': 'gini', 'max_depth': 8, 'min_samples_split': 5, 'max_leaf_nodes': 19}. Best is trial 2 with value: 0.9393333333333334.
[I 2023-07-07 10:51:48,462] Trial 3 finished with value: 0.9393333333333334 and parameters: {'regressor': 'gini', 'max_depth': 9, 'min_samples_split': 2, 'max_leaf_nodes': 14}. Best is trial 2 with value

In [28]:
from sklearn.tree import DecisionTreeClassifier

# [I 2023-07-07 11:00:23,602] Trial 98 finished with value: 0.9396190476190476 and parameters: {'regressor': 'gini', 'max_depth': 5, 'min_samples_split': 9, 'max_leaf_nodes': 26}. Best is trial 87 with value: 0.9397142857142857.

dt = DecisionTreeClassifier(criterion = 'gini',
                            max_depth=5,
                            min_samples_split=9,
                            max_leaf_nodes=26
                           ) # Define classifier
dt.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# Training set performance
dt_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set performance
dt_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % dt_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % dt_test_accuracy)

Model performance for Training set
- Accuracy: 0.9424864283433335
----------------------------------
Model performance for Test set
- Accuracy: 0.9397142857142857


In [17]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    x,y = X_train, y_train
    x_test_rf,y_test_rf = X_test,y_test
    
    n_estimators = trial.suggest_int("n_estimators",3,10)
    max_depth = trial.suggest_int('max_depth',2,6)
    max_features = trial.suggest_int('max_features',3,18)
    
    
    rf = RandomForestClassifier(n_estimators=n_estimators,
                                max_depth=max_depth,
                                max_features=max_features)
        
    rf_cv = cross_val_score(rf, x, y, cv=3,scoring='accuracy') # Train model

    return np.mean(rf_cv)
    
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=99)

[I 2023-07-12 13:18:03,322] A new study created in memory with name: no-name-7682d231-ea3b-402d-aa4b-a2aa0cd87fc3
[I 2023-07-12 13:18:05,672] Trial 0 finished with value: 0.939696465486815 and parameters: {'n_estimators': 7, 'max_depth': 5, 'max_features': 8}. Best is trial 0 with value: 0.939696465486815.
[I 2023-07-12 13:18:07,945] Trial 1 finished with value: 0.9343518378460143 and parameters: {'n_estimators': 10, 'max_depth': 5, 'max_features': 6}. Best is trial 0 with value: 0.939696465486815.
[I 2023-07-12 13:18:09,648] Trial 2 finished with value: 0.9297971394477217 and parameters: {'n_estimators': 7, 'max_depth': 2, 'max_features': 17}. Best is trial 0 with value: 0.939696465486815.
[I 2023-07-12 13:18:10,408] Trial 3 finished with value: 0.9083177868535605 and parameters: {'n_estimators': 3, 'max_depth': 2, 'max_features': 8}. Best is trial 0 with value: 0.939696465486815.
[I 2023-07-12 13:18:11,864] Trial 4 finished with value: 0.9376460108573254 and parameters: {'n_estimator

In [32]:
from sklearn.ensemble import RandomForestClassifier

#I 2023-07-12 13:23:01,168] Trial 95 finished with value: 0.942183902250458 and parameters: {'n_estimators': 10, 'max_depth': 5, 'max_features': 18}. Best is trial 95 with value: 0.942183902250458.

rf = RandomForestClassifier(n_estimators=10,
                           max_depth=5,
                           max_features=18) # Define classifier
rf.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

# Training set performance
rf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set performance
rf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % rf_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % rf_test_accuracy)

Model performance for Training set
- Accuracy: 0.941914990167902
----------------------------------
Model performance for Test set
- Accuracy: 0.9395238095238095


In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import optuna

def objective(trial):
    x,y = X_train, y_train
    
    learning_rate = trial.suggest_float("learning_rate",0.05,0.3)
    max_depth = trial.suggest_int("max_depth",3,10)
    colsample_bytree = trial.suggest_float("colsample_bytree",0,0.8)
    n_estimators = trial.suggest_int("n_estimators",100,2000)
    
    xgb = XGBClassifier(objective='binary:logistic',
                       random_state=42,
                       n_estimators=n_estimators, 
                       learning_rate = learning_rate, 
                       colsample_bytree = colsample_bytree,
                       min_child_weight=1,
                       max_depth=max_depth)

    xgb_cv = cross_val_score(xgb, x, y, cv=3,scoring='accuracy') 

    return np.mean(xgb_cv)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=9)

[I 2023-07-12 11:05:16,302] A new study created in memory with name: no-name-e34794d6-05ff-4b67-b2bf-9042c632e030
[I 2023-07-12 11:13:42,778] Trial 0 finished with value: 0.9410074118892755 and parameters: {'learning_rate': 0.23109533298981327, 'max_depth': 7, 'colsample_bytree': 0.15944325699331446, 'n_estimators': 1395}. Best is trial 0 with value: 0.9410074118892755.
[I 2023-07-12 11:41:53,681] Trial 1 finished with value: 0.94137716600279 and parameters: {'learning_rate': 0.12453724466721904, 'max_depth': 5, 'colsample_bytree': 0.7826026732779335, 'n_estimators': 1761}. Best is trial 1 with value: 0.94137716600279.
[I 2023-07-12 12:11:05,423] Trial 2 finished with value: 0.9416124640750265 and parameters: {'learning_rate': 0.23966118467021447, 'max_depth': 7, 'colsample_bytree': 0.6616242203350557, 'n_estimators': 1802}. Best is trial 2 with value: 0.9416124640750265.
[I 2023-07-12 12:25:03,579] Trial 3 finished with value: 0.9418645691524228 and parameters: {'learning_rate': 0.126

KeyboardInterrupt: 

In [33]:
from xgboost import XGBClassifier

#[I 2023-07-12 12:25:03,579] Trial 3 finished with value: 0.9418645691524228 and parameters: {'learning_rate': 0.12632096102105522, 'max_depth': 8, 'colsample_bytree': 0.33791005980544453, 'n_estimators': 1214}. Best is trial 3 with value: 0.9418645691524228.

xgb = XGBClassifier(objective='binary:logistic',
                    random_state=42,
                    n_estimators=1214, 
                    learning_rate = 0.12632096102105522, 
                    colsample_bytree = 0.33791005980544453,
                    min_child_weight=1,
                    max_depth=8)

xgb.fit(X_train, y_train)

# Make predictions
y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)

# Training set performance
xgb_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set performance
xgb_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % xgb_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % xgb_test_accuracy)

Model performance for Training set
- Accuracy: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.938952380952381


In [14]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=1, max_iter=1000)
mlp.fit(X_train, y_train)

# Make predictions
y_train_pred = mlp.predict(X_train)
y_test_pred = mlp.predict(X_test)

# Training set performance
mlp_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set performance
mlp_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % mlp_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % mlp_test_accuracy)

Model performance for Training set
- Accuracy: 0.9259819492764584
----------------------------------
Model performance for Test set
- Accuracy: 0.9199047619047619


In [18]:
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
import optuna

def objective(trial):
    
    x,y = X_train, y_train
    x_test_cat,y_test_cat = X_test,y_test
    
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.1, log=True)
    depth = trial.suggest_int("depth", 1, 10)
    subsample = trial.suggest_float("subsample", 0.05, 1.0)
    colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.05, 1.0)
    min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 1, 100)

    cat = CatBoostClassifier(iterations=42,
                            learning_rate=learning_rate,
                            depth=depth,
                            subsample=subsample,
                            colsample_bylevel=colsample_bylevel,
                            min_data_in_leaf=min_data_in_leaf)
    
    cat_cv = cross_val_score(cat, x, y, cv=3,scoring='accuracy') 
     
    
    return np.mean(cat_cv)

study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=99)

[I 2023-07-12 13:23:52,200] A new study created in memory with name: no-name-65e9934f-6d43-4329-a14b-fa48955e4ee6


0:	learn: 0.6563962	total: 92.9ms	remaining: 3.81s
1:	learn: 0.6209964	total: 131ms	remaining: 2.63s
2:	learn: 0.5902002	total: 163ms	remaining: 2.12s
3:	learn: 0.5597378	total: 200ms	remaining: 1.9s
4:	learn: 0.5330740	total: 231ms	remaining: 1.71s
5:	learn: 0.5065350	total: 269ms	remaining: 1.61s
6:	learn: 0.4818938	total: 305ms	remaining: 1.53s
7:	learn: 0.4588025	total: 339ms	remaining: 1.44s
8:	learn: 0.4390955	total: 379ms	remaining: 1.39s
9:	learn: 0.4207169	total: 416ms	remaining: 1.33s
10:	learn: 0.4039554	total: 448ms	remaining: 1.26s
11:	learn: 0.3875038	total: 486ms	remaining: 1.21s
12:	learn: 0.3729701	total: 522ms	remaining: 1.16s
13:	learn: 0.3592261	total: 554ms	remaining: 1.11s
14:	learn: 0.3451041	total: 591ms	remaining: 1.06s
15:	learn: 0.3331600	total: 625ms	remaining: 1.01s
16:	learn: 0.3208861	total: 659ms	remaining: 970ms
17:	learn: 0.3098774	total: 699ms	remaining: 932ms
18:	learn: 0.3013662	total: 734ms	remaining: 889ms
19:	learn: 0.2920439	total: 769ms	remaini

[I 2023-07-12 13:23:58,483] Trial 0 finished with value: 0.9415284290492277 and parameters: {'learning_rate': 0.026961153022967622, 'depth': 6, 'subsample': 0.5812734194256836, 'colsample_bylevel': 0.22010097878482965, 'min_data_in_leaf': 94}. Best is trial 0 with value: 0.9415284290492277.


0:	learn: 0.6834585	total: 48.5ms	remaining: 1.99s
1:	learn: 0.6737938	total: 101ms	remaining: 2.01s
2:	learn: 0.6645153	total: 152ms	remaining: 1.98s
3:	learn: 0.6550697	total: 203ms	remaining: 1.93s
4:	learn: 0.6458824	total: 259ms	remaining: 1.92s
5:	learn: 0.6369872	total: 314ms	remaining: 1.88s
6:	learn: 0.6276822	total: 364ms	remaining: 1.82s
7:	learn: 0.6192159	total: 417ms	remaining: 1.77s
8:	learn: 0.6106274	total: 470ms	remaining: 1.72s
9:	learn: 0.6019491	total: 521ms	remaining: 1.67s
10:	learn: 0.5936653	total: 576ms	remaining: 1.62s
11:	learn: 0.5854268	total: 629ms	remaining: 1.57s
12:	learn: 0.5768175	total: 679ms	remaining: 1.51s
13:	learn: 0.5691193	total: 730ms	remaining: 1.46s
14:	learn: 0.5619887	total: 778ms	remaining: 1.4s
15:	learn: 0.5548226	total: 830ms	remaining: 1.35s
16:	learn: 0.5472470	total: 885ms	remaining: 1.3s
17:	learn: 0.5400888	total: 941ms	remaining: 1.25s
18:	learn: 0.5332789	total: 991ms	remaining: 1.2s
19:	learn: 0.5260306	total: 1.04s	remaining

[I 2023-07-12 13:24:06,761] Trial 1 finished with value: 0.9415956570698668 and parameters: {'learning_rate': 0.006910087502469075, 'depth': 7, 'subsample': 0.5673273824110477, 'colsample_bylevel': 0.23292009541319664, 'min_data_in_leaf': 77}. Best is trial 1 with value: 0.9415956570698668.


...

[I 2023-07-12 13:50:26,851] Trial 96 finished with value: 0.9423519723020556 and parameters: {'learning_rate': 0.09070297229252755, 'depth': 8, 'subsample': 0.4548458814558448, 'colsample_bylevel': 0.42779866303238806, 'min_data_in_leaf': 38}. Best is trial 48 with value: 0.942788954436209.


0:	learn: 0.5958165	total: 145ms	remaining: 5.93s
1:	learn: 0.5143189	total: 296ms	remaining: 5.92s
2:	learn: 0.4496893	total: 444ms	remaining: 5.77s
3:	learn: 0.3973079	total: 608ms	remaining: 5.78s
4:	learn: 0.3536661	total: 772ms	remaining: 5.71s
5:	learn: 0.3175174	total: 939ms	remaining: 5.63s
6:	learn: 0.2929480	total: 1.11s	remaining: 5.55s
7:	learn: 0.2698930	total: 1.27s	remaining: 5.42s
8:	learn: 0.2508427	total: 1.44s	remaining: 5.28s
9:	learn: 0.2350679	total: 1.61s	remaining: 5.15s
10:	learn: 0.2232662	total: 1.76s	remaining: 4.97s
11:	learn: 0.2115124	total: 1.93s	remaining: 4.83s
12:	learn: 0.2029039	total: 2.08s	remaining: 4.64s
13:	learn: 0.1946921	total: 2.24s	remaining: 4.47s
14:	learn: 0.1885056	total: 2.4s	remaining: 4.33s
15:	learn: 0.1835378	total: 2.57s	remaining: 4.17s
16:	learn: 0.1791494	total: 2.72s	remaining: 4s
17:	learn: 0.1749096	total: 2.89s	remaining: 3.86s
18:	learn: 0.1714383	total: 3.06s	remaining: 3.71s
19:	learn: 0.1684803	total: 3.22s	remaining: 

[I 2023-07-12 13:50:49,232] Trial 97 finished with value: 0.9422511302710971 and parameters: {'learning_rate': 0.07229033148672168, 'depth': 9, 'subsample': 0.5547465949522173, 'colsample_bylevel': 0.28786579435287296, 'min_data_in_leaf': 46}. Best is trial 48 with value: 0.942788954436209.


0:	learn: 0.5867617	total: 76.1ms	remaining: 3.12s
1:	learn: 0.5043015	total: 154ms	remaining: 3.08s
2:	learn: 0.4385183	total: 234ms	remaining: 3.04s
3:	learn: 0.3837223	total: 313ms	remaining: 2.98s
4:	learn: 0.3417542	total: 392ms	remaining: 2.9s
5:	learn: 0.3058792	total: 476ms	remaining: 2.85s
6:	learn: 0.2781312	total: 561ms	remaining: 2.81s
7:	learn: 0.2574408	total: 645ms	remaining: 2.74s
8:	learn: 0.2400083	total: 728ms	remaining: 2.67s
9:	learn: 0.2255215	total: 809ms	remaining: 2.59s
10:	learn: 0.2157676	total: 888ms	remaining: 2.5s
11:	learn: 0.2078416	total: 968ms	remaining: 2.42s
12:	learn: 0.2017669	total: 1.05s	remaining: 2.34s
13:	learn: 0.1940965	total: 1.13s	remaining: 2.27s
14:	learn: 0.1892605	total: 1.21s	remaining: 2.18s
15:	learn: 0.1849177	total: 1.3s	remaining: 2.11s
16:	learn: 0.1815733	total: 1.38s	remaining: 2.03s
17:	learn: 0.1762117	total: 1.46s	remaining: 1.95s
18:	learn: 0.1732210	total: 1.55s	remaining: 1.88s
19:	learn: 0.1704108	total: 1.63s	remaining

[I 2023-07-12 13:51:01,713] Trial 98 finished with value: 0.942318358291736 and parameters: {'learning_rate': 0.07946442731396093, 'depth': 7, 'subsample': 0.5102180507190525, 'colsample_bylevel': 0.46711450467231347, 'min_data_in_leaf': 36}. Best is trial 48 with value: 0.942788954436209.


In [34]:
from catboost import CatBoostClassifier

#[I 2023-07-12 13:32:18,501] Trial 48 finished with value: 0.942788954436209 and parameters: {'learning_rate': 0.09232052445789558, 'depth': 6, 'subsample': 0.6643962603667595, 'colsample_bylevel': 0.3791180404872756, 'min_data_in_leaf': 50}. Best is trial 48 with value: 0.942788954436209.

cat = CatBoostClassifier(iterations=999,
                         learning_rate=0.09232052445789558, 
                         depth=6, 
                         subsample=0.6643962603667595, 
                         colsample_bylevel=0.3791180404872756, 
                         min_data_in_leaf=50,
                         verbose=False)


cat.fit(X_train, y_train,
        eval_set=(X_test, y_test))

y_test_pred = cat.predict(X_test)
cat_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Test set')
print('- Accuracy: %s' % cat_test_accuracy)

Model performance for Test set
- Accuracy: 0.9393333333333334


In [51]:
from lightgbm import LGBMClassifier
import optuna

def objective(trial):
    
    x,y = X_train, y_train
    # x_test_lgb,y_test_lgb = X_test,y_test
    
    n_estimators = trial.suggest_categorical("n_estimators", [999])
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
    num_leaves = trial.suggest_int("num_leaves", 20, 3000, step=20)
    max_depth = trial.suggest_int("max_depth", 3, 12)
    min_data_in_leaf  = trial.suggest_int("min_data_in_leaf", 200, 10000, step=100)
    lambda_l1 = trial.suggest_int("lambda_l1", 0, 100, step=5)
    lambda_l2 = trial.suggest_int("lambda_l2", 0, 100, step=5)
    min_gain_to_split = trial.suggest_float("min_gain_to_split", 0, 15)
    bagging_fraction = trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1)
    bagging_freq = trial.suggest_categorical("bagging_freq", [1])
    feature_fraction = trial.suggest_float("feature_fraction", 0.2, 0.95, step=0.1)

    lgb = LGBMClassifier(objective="binary", 
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        num_leaves=num_leaves,
                        max_depth=max_depth,
                        min_data_in_leaf=min_data_in_leaf,
                        lambda_l1=lambda_l1,
                        lambda_l2=lambda_l1,
                        min_gain_to_split=min_gain_to_split,
                        bagging_fraction=bagging_fraction,
                        bagging_freq=bagging_freq,
                        feature_fraction=feature_fraction)
                                           
                                           
                                           
    lgb_cv = cross_val_score(lgb, x, y, cv=3,scoring='accuracy') 
     
    
    return np.mean(lgb_cv)

study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=21)

[I 2023-07-12 17:59:08,309] A new study created in memory with name: no-name-6a079d3d-e501-491e-8fc8-09ef086c7870
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] min_gain_to_split is set=6.772404150034722, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.772404150034722
[LightGBM] [Warning] lambda_l2 is set=20, reg_lambda=0.0 will be ignored. Current value: lambda_l2=20
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=7600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7600
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will 

[I 2023-07-12 17:59:26,614] Trial 0 finished with value: 0.9340493117531387 and parameters: {'n_estimators': 999, 'learning_rate': 0.06590975104735797, 'num_leaves': 2840, 'max_depth': 8, 'min_data_in_leaf': 7600, 'lambda_l1': 20, 'lambda_l2': 5, 'min_gain_to_split': 6.772404150034722, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 0.9340493117531387.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=4.01434607247872, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.01434607247872
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=2100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction i

[I 2023-07-12 18:00:20,630] Trial 1 finished with value: 0.942788954436209 and parameters: {'n_estimators': 999, 'learning_rate': 0.25096854082761894, 'num_leaves': 3000, 'max_depth': 5, 'min_data_in_leaf': 2100, 'lambda_l1': 10, 'lambda_l2': 30, 'min_gain_to_split': 4.01434607247872, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.942788954436209.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[I 2023-07-12 18:11:37,919] Trial 18 finished with value: 0.9412763239718315 and parameters: {'n_estimators': 999, 'learning_rate': 0.18606760753980586, 'num_leaves': 2680, 'max_depth': 10, 'min_data_in_leaf': 300, 'lambda_l1': 95, 'lambda_l2': 40, 'min_gain_to_split': 0.010281345522779262, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.942788954436209.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=55, reg_alpha=0.0 will be ignored. Current value: lambda_l1=55
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=2.1107494503793647, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.1107494503793647
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=1400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=55, reg_alpha=0.0 will be ignored. Current value: lambda_l1=55
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 wil

[I 2023-07-12 18:12:39,654] Trial 19 finished with value: 0.9413267449873107 and parameters: {'n_estimators': 999, 'learning_rate': 0.11856059156197721, 'num_leaves': 2540, 'max_depth': 9, 'min_data_in_leaf': 1400, 'lambda_l1': 55, 'lambda_l2': 55, 'min_gain_to_split': 2.1107494503793647, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 1 with value: 0.942788954436209.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=80, reg_alpha=0.0 will be ignored. Current value: lambda_l1=80
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[LightGBM] [Warning] min_gain_to_split is set=2.935234100949982, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.935234100949982
[LightGBM] [Warning] lambda_l2 is set=80, reg_lambda=0.0 will be ignored. Current value: lambda_l2=80
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=80, reg_alpha=0.0 will be ignored. Current value: lambda_l1=80
[LightGBM] [Warning] bagging_fraction is 

[I 2023-07-12 18:13:24,549] Trial 20 finished with value: 0.9397636935074539 and parameters: {'n_estimators': 999, 'learning_rate': 0.23442235313478113, 'num_leaves': 2180, 'max_depth': 12, 'min_data_in_leaf': 300, 'lambda_l1': 80, 'lambda_l2': 15, 'min_gain_to_split': 2.935234100949982, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 1 with value: 0.942788954436209.


In [52]:
from lightgbm import LGBMClassifier

#Trial 1 finished with value: 0.942788954436209 and parameters: {'n_estimators': 999, 
#'learning_rate': 0.25096854082761894, 'num_leaves': 3000, 'max_depth': 5, 'min_data_in_leaf': 2100, 
#                'lambda_l1': 10, 'lambda_l2': 30, 'min_gain_to_split': 4.01434607247872, 'bagging_fraction': 0.7, 
#                                'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.942788954436209.

lgb = LGBMClassifier(objective="binary", 
                        n_estimators=999,
                        learning_rate=0.25096854082761894,
                        num_leaves=3000,
                        max_depth=5,
                        min_data_in_leaf=2100,
                        lambda_l1=10,
                        lambda_l2=30,
                        min_gain_to_split=4.01434607247872,
                        bagging_fraction=0.7,
                        bagging_freq=1,
                        feature_fraction=0.30000000000000004)
                                           
                                           
                                           
lgb.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric="accuracy")
                                    
        
y_test_pred = lgb.predict(X_test)
lgb_test_accuracy = accuracy_score(y_test, y_test_pred)  

print('Model performance for Test set')
print('- Accuracy: %s' % lgb_test_accuracy)

[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=4.01434607247872, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.01434607247872
[LightGBM] [Warning] lambda_l2 is set=30, reg_lambda=0.0 will be ignored. Current value: lambda_l2=30
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=2100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.250018
[2]	valid_0's binary_logloss: 0.228596
[3]	valid_0's binary_logloss: 0.215015
[4]	valid_0's binary_lo

In [23]:
# STACKING ENSEMBLE
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    # ('knn',knn),
    # ('svm_rbf',svm_rbf),
    # ('dt',dt),
    ('rf',rf),
    ('xgb',xgb),
    ('cat',cat)
    # ('lgb',lgb)
    #('mlp',mlp) 
]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, 
    final_estimator=LogisticRegression(),
    cv = 9
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)

Model performance for Training set
- Accuracy: 0.9586211532966941
----------------------------------
Model performance for Test set
- Accuracy: 0.9391428571428572


In [53]:
# VOTING ENSEMBLE
from sklearn.ensemble import VotingClassifier

estimator_list = [
    # ('knn',knn),
    # ('svm_rbf',svm_rbf),
    # ('dt',dt),
    ('rf',rf),
    ('xgb',xgb),
    ('cat',cat),
    ('lgb',lgb)
    #('mlp',mlp) 
]

# Build model
voting_model = VotingClassifier(estimators=estimator_list,
                                voting='soft',
                                weights=[3,1,2,1])


# Train stacked model
voting_model.fit(X_train, y_train)

# Make predictions
y_train_pred = voting_model.predict(X_train)
y_test_pred = voting_model.predict(X_test)

# Training set model performance
voting_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy

# Test set model performance
voting_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy

print('Model performance for Training set')
print('- Accuracy: %s' % voting_model_train_accuracy)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % voting_model_test_accuracy)

[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=4.01434607247872, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.01434607247872
[LightGBM] [Warning] lambda_l2 is set=30, reg_lambda=0.0 will be ignored. Current value: lambda_l2=30
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=2100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Model performance for Training set
- Accuracy: 0.9599657137094741
----------------------------------
Model performance for Test set
- Accura

In [54]:
acc_test_list = {
'rf': rf_test_accuracy,
'xgb' : xgb_test_accuracy,
'cat' : cat_test_accuracy,    
'lgb' : lgb_test_accuracy,
'stack': stack_model_test_accuracy,
'voting': voting_model_test_accuracy}

acc_df = pd.DataFrame.from_dict(acc_test_list, orient='index', columns=['Accuracy'])
acc_df

,Accuracy
rf,0.939524
xgb,0.938952
cat,0.939333
lgb,0.939143
stack,0.939143
voting,0.940190


In [55]:
prediction = pd.DataFrame(voting_model.predict(test).round())

id_test = pd.read_csv('/kaggle/input/telecom-churn/test.csv')
id_test = id_test['id']


submission = pd.concat([id_test,prediction], axis=1)
submission = submission.astype({0:'int'})
submission.rename(columns = {0:'churn_probability'},inplace=True)
submission.to_csv(r"submission15.csv", index=False)

In [ ]:
# use cv for hyperparameter tuning on CAT,XGB,RF
# how to weight the model on stacking phase?
# passthrough argument on stacking regressor?